# Mergin Maps Projects Management

Mergin Maps API allows you to manage your projects in a simple and effective way. See the [API reference](https://merginmaps.com/docs/dev/integration/) for more details about methods used in this notebook.

First let's install mergin maps client (if not installed yet)

In [ ]:
!pip install mergin-client

Login to Mergin Maps using your an existing user

In [ ]:
# Use here your login username and password
LOGIN="..."
PASSWORD="..."

import mergin

client = mergin.MerginClient(login=LOGIN, password=PASSWORD)

Now you can use the client to call the API. Let's try to clone the project available for this example (`lutraconsulting/Vienna trees example`) to your Mergin Maps projects. 

You need to specify a workspace and project where our sample project will be cloned to. We can split our template to two projects.

In [ ]:
WORKSPACE = "..."
PROJECT = "project"

client.clone_project(
    source_project_path=f"lutraconsulting/Vienna trees example", cloned_project_name=f"{WORKSPACE}/{PROJECT}-team-a"
)
client.clone_project(
    source_project_path=f"lutraconsulting/Vienna trees example", cloned_project_name=f"{WORKSPACE}/{PROJECT}-team-b"
)

Download locally your newly created Mergin Maps project to add prepare data for your teams (Team A and Team B).

In [7]:
# download project to local folder.
LOCAL_FOLDER_TEAM_A="/tmp/project-team-a"
LOCAL_FOLDER_TEAM_B="/tmp/project-team-b"
client.download_project(project_path=f"{WORKSPACE}/{PROJECT}-team-a", directory=LOCAL_FOLDER_TEAM_A)
client.download_project(project_path=f"{WORKSPACE}/{PROJECT}-team-b", directory=LOCAL_FOLDER_TEAM_B)

Let's add 20 trees to the geopackage for each team. See geopackages prepared for this example in the [./03_projects_assets](./03_projects_assets) folder.

In [ ]:
# copy geopackage for team A to .gpkg file in LOCAL_FOLDER_TEAM_A
import shutil
shutil.copyfile(
    f"./03_projects_assets/Ready_to_survey_trees_team_A.gpkg",
    f"{LOCAL_FOLDER_TEAM_A}/Ready_to_survey_trees.gpkg"
)
# copy geopackage for team B to .gpkg file in LOCAL_FOLDER_TEAM_B
import shutil
shutil.copyfile(
    f"./03_projects_assets/Ready_to_survey_trees_team_B.gpkg",
    f"{LOCAL_FOLDER_TEAM_B}/Ready_to_survey_trees.gpkg"
)

'/tmp/project-team-b/Ready_to_survey_trees.gpkg'

If you open your QGIS with projects for teams, you can see that every team is having different points to check.

<img src="./03_projects_assets/team_a.png" alt="Team A" style="width: 200px;"/> <img src="./03_projects_assets/team_b.png" alt="Team B" style="width: 200px;"/>

Let's sync your changes with Mergin Maps.

In [11]:
client.push_project(directory=f"{LOCAL_FOLDER_TEAM_A}")
client.push_project(directory=f"{LOCAL_FOLDER_TEAM_B}")

If you have prepared data for your teams, you can now add some surveyors to projects. We can use the guest user role for this, as it only grants access to a specific projects. Let's add `john.doeA@example.com` and  `john.doeB@example.com` to the teams.

In [12]:
from mergin.common import WorkspaceRole

# First, let's get workspace ID
WORKSPACE_ID = None
for p in client.workspaces_list():
  if p['name'] == WORKSPACE:
    WORKSPACE_ID = p['id']

print(f"WORKSPACE_ID: {WORKSPACE_ID}")

user_team_a = client.create_user(
    email="john.doeA@example.com",
    password="JohnDoe123",
    workspace_id=WORKSPACE_ID,
    workspace_role=WorkspaceRole.GUEST
)
user_team_b = client.create_user(
    email="john.doeB@example.com",
    password="JohnDoe123",
    workspace_id=WORKSPACE_ID,
    workspace_role=WorkspaceRole.GUEST
)

WORKSPACE_ID: 60538


Created users do not have any permissions now. Let's add users as collaborators to the projects. They will be able to start the survey with Mergin Maps mobile application.

In [ ]:
from mergin.common import ProjectRole

team_a_project = client.project_info(f"{WORKSPACE}/{PROJECT}-team-a")
team_b_project = client.project_info(f"{WORKSPACE}/{PROJECT}-team-b")

# Add users to the projects
client.add_project_collaborator(
    project_id=team_a_project.get("id"), user="john.doeA@example.com", project_role=ProjectRole.EDITOR
)
client.add_project_collaborator(
    project_id=team_b_project.get("id"), user="john.doeB@example.com", project_role=ProjectRole.EDITOR
)

{'email': 'john.doeB@example.com',
 'id': 122393,
 'project_role': 'editor',
 'role': 'editor',
 'username': 'john.doeb',
 'workspace_role': 'guest'}

Let's delete cloned projects to make cleanup after the example.

NOTE: using `delete_project_now` will bypass the default value `DELETED_PROJECT_EXPIRATION`. See: https://merginmaps.com/docs/server/environment/#data-synchronisation-and-management

In [16]:
client.delete_project_now(f"{WORKSPACE}/{PROJECT}-team-a")
client.delete_project_now(f"{WORKSPACE}/{PROJECT}-team-b")